In [1]:
import pandas as pd
import os
import numpy as np
import re

# Dataset:  Cornell Movie--Dialogs Corpus

* contains a large metadata-rich collection of fictional conversations extracted from raw movie scripts:
  - 220,579 conversational exchanges between 10,292 pairs of movie characters
* website: http://www.cs.cornell.edu/~cristian/Cornell_Movie-Dialogs_Corpus.html

In [2]:
datadir = '/data/cornell-movie-dialogs-corpus'

In [3]:
# Files included in the zip file
os.listdir(datadir)

['.DS_Store',
 'chameleons.pdf',
 'movie_characters_metadata.txt',
 'movie_conversations.txt',
 'movie_lines.txt',
 'movie_titles_metadata.txt',
 'raw_script_urls.txt',
 'README.txt']

In [4]:
# Extract the conversation lines
f = open(os.path.join(datadir, 'movie_conversations.txt'), 'rb')
conversations = []
for line in f.readlines(): 
    conversations.append([x.strip('\'') for x in line.decode('windows-1252').strip().split(" +++$+++ ")[-1][1:-1].split(", ")])    

In [5]:
# Clean the text

def clean_text(text):
    '''Clean text by removing unnecessary characters and altering the format of words.'''

    text = text.lower()
    
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    
    return text

In [6]:
# Example of cleaning text
clean_text("Just once.  Afterwards, I told him I didn't want to anymore. I wasn't ready. He got pissed.  Then he broke up with me.")

'just once  afterwards i told him i did not want to anymore i was not ready he got pissed  then he broke up with me'

In [7]:
# Extract the sentense and line number pairs
f = open(os.path.join(datadir, 'movie_lines.txt'), 'rb')
movie_lines = {}
for line in f.readlines():
    fields = line.decode('windows-1252').strip().split(" +++$+++ ")
    movie_lines[fields[0]] = clean_text(fields[-1])

In [8]:
# Tokenizer

# Regular expressions used to tokenize.
_WORD_SPLIT = re.compile("([.,!?\"':;)(])")
# _DIGIT_RE = re.compile(r"\d")

def basic_tokenizer(line):    
    # break line to sentences
    sentences = re.split(_WORD_SPLIT, line)
    words = []
    for sentence in sentences:
        if sentence:
            words += sentence.strip().split()
    return words


line = 'just once  afterwards i told him i did not want to anymore  i was not ready he got pissed  then he broke up with me'
tokens =  basic_tokenizer(line)
tokens

['just',
 'once',
 'afterwards',
 'i',
 'told',
 'him',
 'i',
 'did',
 'not',
 'want',
 'to',
 'anymore',
 'i',
 'was',
 'not',
 'ready',
 'he',
 'got',
 'pissed',
 'then',
 'he',
 'broke',
 'up',
 'with',
 'me']

In [9]:
# extrac the Q_A pairs, only use the sentences with length <=10 and >=2
questions = []
answers=[]
min_length = 2
enc_sentence_length = 10
dec_sentence_length = 10
for c in conversations:
    n = len(c)
    for i in range(0,n-1):
        
        qtokens = basic_tokenizer(movie_lines[c[i]])
        atokens =  basic_tokenizer(movie_lines[c[i+1]])
       
        if len(qtokens)<=enc_sentence_length and len(qtokens)>=min_length \
        and len(atokens)<=dec_sentence_length and len(atokens)>=min_length :
            questions.append(movie_lines[c[i]])
            answers.append(movie_lines[c[i+1]])
            
len(questions), len(answers)
       

(73358, 73358)

In [10]:
#split into to trainset and testset
from sklearn.model_selection import train_test_split
Q_train, Q_test, A_train, A_test = train_test_split(  questions, answers, test_size=0.2, random_state=42)

In [11]:
#Generate the vocabulary list for encoder and decoder

_PAD = "_PAD"
_GO = "_GO"
_EOS = "_EOS"
_UNK = "_UNK"
_START_VOCAB = [ _GO, _PAD, _EOS, _UNK]

from tqdm import tqdm
def get_vocab(train, max_vocab_size):
    vocab = {}
    for line in tqdm(train):
        tokens = basic_tokenizer(line)
        for word in tokens:
#             word = re.sub(_DIGIT_RE, "0", w) 
            if word in vocab:
                vocab[word] += 1
            else:
                vocab[word] = 1
    
    vocab_list = _START_VOCAB + sorted(vocab, key=vocab.get, reverse=True)
    
    print("total vocab = ", len(vocab_list))
    print('vocab with freq = 1: ', list(vocab.values()).count(1))
    print('vocab with freq = 2: ', list(vocab.values()).count(2))
    print('vocab with freq = 3: ', list(vocab.values()).count(3))
    vocab_list = vocab_list[:max_vocab_size]
    
    return vocab_list


# only choose part of the vocab for the training
max_encode_vocab_size = 2000
max_decode_vocab_size = 2000
encode_vocab = get_vocab(Q_train, max_encode_vocab_size)
decode_vocab = get_vocab(A_train, max_encode_vocab_size)

 47%|████▋     | 27556/58686 [00:00<00:00, 275553.69it/s]

total vocab =  17415
vocab with freq = 1:  9186
vocab with freq = 2:  2734
vocab with freq = 3:  1271


100%|██████████| 58686/58686 [00:00<00:00, 281398.25it/s]

total vocab =  17474
vocab with freq = 1:  9492
vocab with freq = 2:  2623
vocab with freq = 3:  1211


# Padding

Convert the variable length sequences into fixed length sequences, by padding. We use a few special symbols to fill in the sequence.

* EOS : End of sentence
* PAD : Filler
* GO : Start decoding
* UNK : Unknown; word not in vocabulary


In [12]:


GO_ID = 0
PAD_ID = 1
EOS_ID = 2
UNK_ID = 3

dec_sentence_length = 12
enc_sentence_length = 12

def token2idx(word, vocab):
    return vocab.get(word, UNK_ID)

def sent2idx(sent, vocab=encode_vocab, max_sentence_length=enc_sentence_length, is_target=False):
    tokens = basic_tokenizer(sent)
    current_length = len(tokens)
    vocab = dict([(x, y) for (y, x) in enumerate(vocab)])
    
    pad_length = max_sentence_length - current_length
    if is_target:
        pad_length -= 2
        return [0] + [token2idx(token, vocab) for token in tokens] + [2] + [1] * pad_length
    else:
        return [token2idx(token, vocab) for token in tokens] + [0] * pad_length

# Enc Example
print('Hi What is your name?')
print(sent2idx('Hi What is your name?'))

# Dec Example
print('Hi this is Jaemin.')
print(sent2idx('Hi this is Jaemin.', vocab=decode_vocab, max_sentence_length=dec_sentence_length, is_target=True))

Hi What is your name?
[3, 3, 6, 28, 124, 3, 0, 0, 0, 0, 0, 0]
Hi this is Jaemin.
[0, 3, 32, 6, 3, 3, 2, 1, 1, 1, 1, 1]


In [13]:
PAD_ID = 0
GO_ID = 1
EOS_ID = 2
UNK_ID = 3

import random
import numpy as np

def get_batch(Q_train, A_train, batch_size):
    
    encoder_inputs = []
    decoder_inputs = []
    for _ in range(batch_size):
        id = random.choice(range(len(Q_train)))
        encoder_input, decoder_input = Q_train[id], A_train[id]
        
        encoder_inputs.append(sent2idx(encoder_input, max_sentence_length=enc_sentence_length))
        decoder_inputs.append(sent2idx(decoder_input, vocab=decode_vocab, 
                                       max_sentence_length=dec_sentence_length, is_target=True))


    return np.array(encoder_inputs), np.array(decoder_inputs)

# example of small batches
batch_encoder_inputs, batch_decoder_inputs= get_batch(Q_train, A_train, 5)
batch_encoder_inputs, batch_decoder_inputs 

(array([[   4,   55,    4,   18,  321,   28,  486,    0,    0,    0,    0,
            0],
        [ 295,   66,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0],
        [   3,    9,  679,   20,  151,   13, 1690,    0,    0,    0,    0,
            0],
        [   5,   29,    8,   18,   21,   23,   32,  166,    0,    0,    0,
            0],
        [  14,   10,    4,   83,    8,  335,   28,    3,    0,    0,    0,
            0]]),
 array([[   0,  110,  110,    9,  133,   24,    9, 1989,    2,    1,    1,
            1],
        [   0,  147,   35,   11,  129,   83,    9,  515,  619,    3,    2,
            1],
        [   0,   40,    5,   25,   10,   28,  333,    3,    2,    1,    1,
            1],
        [   0,    5,   34,   60,   12,  211,    2,    1,    1,    1,    1,
            1],
        [   0,   59,   49,    4,   80,    2,    1,    1,    1,    1,    1,
            1]]))

In [14]:
# convert the id to token
def idx2token(idx, reverse_vocab):
    return reverse_vocab[idx]

def idx2sent(indices, reverse_vocab):
    return " ".join([idx2token(idx, reverse_vocab) for idx in indices])

In [15]:
idx2sent( [0, 29, 58, 6, 38, 3, 13, 3, 29, 1, 1], reverse_vocab = decode_vocab)

'_GO your go is about _UNK that _UNK your _PAD _PAD'

# Model

seq2seq model

<img style="float: left;" src="attention.png">

In [16]:
import tensorflow as tf
import data_utils
import seq2seq_model
layer_size = 512
num_layers = 2
max_gradient_norm = 5.0
learning_rate = 0.0001
learning_rate_decay_factor = 0.9
min_learning_rate = 0.000001
batch_size = 64

enc_sentence_length = 12
dec_sentence_length = 12
keep_prob = 0.75

enc_emb_size = layer_size
dec_emb_size = layer_size

In [17]:
tf.reset_default_graph()

enc_inputs = tf.placeholder(
    tf.int32,
    shape=[None, enc_sentence_length],
    name='input_sentences')


dec_inputs = tf.placeholder(
    tf.int32,
    shape=[None, dec_sentence_length],
    name='output_sentences')

# batch_major => time_major
enc_inputs_t = tf.transpose(enc_inputs, [1,0])
dec_inputs_t = tf.transpose(dec_inputs, [1,0])


lstm = tf.contrib.rnn.BasicLSTMCell(layer_size)
drop = tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob = keep_prob)
cell = tf.contrib.rnn.MultiRNNCell([drop] * num_layers)
# cell = tf.nn.rnn_cell.BasicRNNCell(layer_size)

with tf.variable_scope("embedding_attention_seq2seq"):
    # dec_outputs: [dec_sent_len+1 x batch_size x hidden_size]
    dec_outputs, dec_last_state = tf.contrib.legacy_seq2seq.embedding_attention_seq2seq(
        encoder_inputs=tf.unstack(enc_inputs_t),
        decoder_inputs=tf.unstack(dec_inputs_t),
        cell=cell, 
        num_encoder_symbols=max_encode_vocab_size,
        num_decoder_symbols=max_decode_vocab_size,
        embedding_size=enc_emb_size,
        feed_previous=True)

# predictions: [batch_size x dec_sentence_lengths]
predictions = tf.transpose(tf.argmax(tf.stack(dec_outputs), axis=-1), [1,0])

# labels & logits: [dec_sentence_length x batch_size x dec_vocab_size]
labels = tf.one_hot(dec_inputs_t, max_decode_vocab_size)
logits = tf.stack(dec_outputs)



with tf.name_scope("optimization"):
    # Loss function
    loss = tf.contrib.seq2seq.sequence_loss([logits], [labels], [tf.ones_like(labels, dtype=tf.float32)])
    #     loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits( labels=labels, logits=logits))
    # Optimizer
    optimizer = tf.train.AdamOptimizer(learning_rate)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(loss)
    capped_gradients = [(tf.clip_by_value(grad, -5., 5.), var) for grad, var in gradients if grad is not None]
    training_op = optimizer.apply_gradients(capped_gradients)

In [18]:
import tensorflow as tf
max_steps=50000
model_save_path = './lstm_model.ckpt'

init = tf.global_variables_initializer()
saver = tf.train.Saver()



with tf.Session() as sess:

#     saver.restore(sess, model_save_path)
    sess.run(init)
    
    for step in tqdm(range(max_steps)):
        batch_encoder_inputs, batch_decoder_inputs= get_batch(Q_train, A_train, batch_size)
        
        
        batch_preds, batch_loss, _ = sess.run(
                [predictions, loss, training_op],
                feed_dict={
                    enc_inputs: batch_encoder_inputs,
                    dec_inputs: batch_decoder_inputs
                })
 
        
        if step % 1000 == 0:
            print('step', step) 
            batch_encoder_inputs, batch_decoder_inputs= get_batch(Q_test, A_test, 4)
             
            preds = sess.run(predictions, feed_dict={
                    enc_inputs: batch_encoder_inputs,
                    dec_inputs: batch_decoder_inputs
                })
            
            
           
            for input_sent, target_sent, pred in zip( batch_encoder_inputs, batch_decoder_inputs, preds):
                print('\t', idx2sent(input_sent, reverse_vocab = encode_vocab))
                print('\t => ', idx2sent(pred, reverse_vocab=decode_vocab))
                print('\tCorrent answer:', idx2sent(target_sent, reverse_vocab = decode_vocab))
            print('\tepoch loss: {:.2f}\n'.format(batch_loss))
            
            save_path = saver.save(sess, save_path= model_save_path)
            print("Model saved in file: %s" % save_path) 
            
             learning_rate = learning_rate * learning_rate_decay_factor
            if learning_rate < min_learning_rate:
                learning_rate = min_learning_rate



  0%|          | 0/50000 [00:00<?, ?it/s]

step 0
	 he just asked me questions _GO _GO _GO _GO _GO _GO _GO
	 =>  _PAD _PAD _PAD _PAD _PAD _PAD _PAD _PAD _PAD _PAD _PAD _PAD
	Corrent answer: _GO what _UNK of questions _EOS _PAD _PAD _PAD _PAD _PAD _PAD
	 yes that is up _GO _GO _GO _GO _GO _GO _GO _GO
	 =>  you _PAD _PAD _PAD _PAD _PAD _PAD _PAD _PAD _PAD _PAD _PAD
	Corrent answer: _GO nothing calling to say hi that is up with you _EOS
	 but i have to finish this _GO _GO _GO _GO _GO _GO
	 =>  shooting shooting _PAD _PAD _PAD _PAD _PAD _PAD _PAD _PAD _PAD _PAD
	Corrent answer: _GO come on that shit can wait _EOS _PAD _PAD _PAD _PAD
	 what was that _GO _GO _GO _GO _GO _GO _GO _GO _GO
	 =>  like _PAD _PAD _PAD _PAD _PAD _PAD _PAD _PAD _PAD _PAD _PAD
	Corrent answer: _GO his horse _EOS _PAD _PAD _PAD _PAD _PAD _PAD _PAD _PAD
	epoch loss: 7.60



  0%|          | 1/50000 [00:03<53:49:33,  3.88s/it]

Model saved in file: ./lstm_model.ckpt


  2%|▏         | 1000/50000 [24:28<20:07:05,  1.48s/it]

...

 98%|█████████▊| 49000/50000 [22:09:55<1:00:36,  3.64s/it]

step 49000
	 i am not finished yet _GO _GO _GO _GO _GO _GO _GO
	 =>  _GO not with are _EOS _EOS _PAD _PAD _PAD _PAD _PAD _PAD
	Corrent answer: _GO go on _EOS _PAD _PAD _PAD _PAD _PAD _PAD _PAD _PAD
	 where did you come from _GO _GO _GO _GO _GO _GO _GO
	 =>  _GO i am _UNK _UNK _UNK _UNK _UNK _UNK _UNK _EOS _PAD
	Corrent answer: _GO do you believe in time _UNK _UNK _EOS _PAD _PAD _PAD
	 great _UNK _GO _GO _GO _GO _GO _GO _GO _GO _GO _GO
	 =>  _GO yeah _UNK _EOS _PAD _PAD _PAD _PAD _PAD _PAD _PAD _PAD
	Corrent answer: _GO how is the store _EOS _PAD _PAD _PAD _PAD _PAD _PAD
	 i am a _UNK just like you man _GO _GO _GO _GO
	 =>  _GO i i have been to to _UNK _EOS _PAD _PAD _PAD
	Corrent answer: _GO what do you mean _EOS _PAD _PAD _PAD _PAD _PAD _PAD
	epoch loss: 1.57



 98%|█████████▊| 49001/50000 [22:10:05<1:28:39,  5.32s/it]

Model saved in file: ./lstm_model.ckpt


100%|██████████| 50000/50000 [23:11:47<00:00,  3.59s/it]  


# Improvement

* more epochs/steps
* bigger vocabulary (2000-->20000)
* word2vec embedding
* bucket model